In [ ]:
import pandas as pd
from google.colab import files


# -----------------------------
# Step 1: Define file paths and read data
# -----------------------------
####AND
# -----------------------------
# Step 2: Read the Provider sheet (already done in Step 1)
# -----------------------------
# The actual table starts at row 13 (header=12) in these NHS files
# df1 = pd.read_excel(file1, sheet_name="Provider", header=12) # Removed redundant line
# df2 = pd.read_excel(file2, sheet_name="Provider", header=12) # Removed redundant line
# df3 = pd.read_excel(file3, sheet_name="Provider", header=12) # Removed redundant line


# Modality Provider Counts
url = "https://docs.google.com/spreadsheets/d/1NVr9HQbkIpyiQcEAyguJRK-xNFqRQ4oC/edit?usp=sharing&ouid=117443651029460089438&rtpof=true&sd=true"
path1 = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
df1 = pd.read_excel(path1, sheet_name="Provider", header=12)

# Modality Provider Request to Test
url = "https://docs.google.com/spreadsheets/d/1MTrOXWxLwRuFYdDyeeiYZWuxos4M-pzy/edit?usp=sharing&ouid=117443651029460089438&rtpof=true&sd=true"
path2 = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
df2 = pd.read_excel(path2, sheet_name="Provider", header=12)

# Modality Provider Test to Report
url = "https://docs.google.com/spreadsheets/d/1bFPrmO8l0P2XBH6u9ZUGoM12KjnoKp6F/edit?usp=sharing&ouid=117443651029460089438&rtpof=true&sd=true"
path3 = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
df3 = pd.read_excel(path3, sheet_name="Provider", header=12)




# -----------------------------
# Step 3: Select key columns and renamed date columns in all three dataframes
# -----------------------------
cols_to_keep = ["Region", "Org Code", "Provider name", "Modality", "Year to date"]

t1 = df1[cols_to_keep].rename(columns={"Year to date": "Total_Scans"})
t2 = df2[cols_to_keep].rename(columns={"Year to date": "RequestToTest_Days"})
t3 = df3[cols_to_keep].rename(columns={"Year to date": "TestToReport_Days"})

# -----------------------------
# Step 4: Merge all tables
# -----------------------------
merge_keys = ["Region", "Org Code", "Provider name", "Modality"]

merged = (
    t1.merge(t2[merge_keys + ["RequestToTest_Days"]], on=merge_keys, how="left")
      .merge(t3[merge_keys + ["TestToReport_Days"]], on=merge_keys, how="left")
)

# -----------------------------
# Step 5: Clean numeric values
# -----------------------------
for col in ["Total_Scans", "RequestToTest_Days", "TestToReport_Days"]:
    merged[col] = (
        merged[col].astype(str)
        .str.replace(",", "", regex=False)
        .str.replace("-", "", regex=False)
        .replace("nan", None)
    )
    merged[col] = pd.to_numeric(merged[col], errors="coerce")

# -----------------------------
# Step 6: Create derived metric (e.g.,Total_Turnaround_Days would represent the total time from when your doctor requests the X-ray to when the final report from the radiologist is ready)
# -----------------------------
merged["Total_Turnaround_Days"] = (
    merged["RequestToTest_Days"] + merged["TestToReport_Days"]
)

numeric_columns = ['Total_Scans', 'RequestToTest_Days', 'TestToReport_Days', 'Total_Turnaround_Days']

for col in numeric_columns:
    merged[col] = pd.to_numeric(merged[col], errors='coerce')  # Convert to numeric, invalid -> NaN


# -----------------------------
# Step 7: Export clean dataset
# -----------------------------
merged.to_csv("diagnostic_imaging_tidy.csv", index=False, quoting=1)
files.download("diagnostic_imaging_tidy.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
merged.head(50)

,Region,Org Code,Provider name,Modality,Total_Scans,RequestToTest_Days,TestToReport_Days,Total_Turnaround_Days
0,-,-,ENGLAND,Computerized Axial Tomography,8291725.0,1.0,0.0,1.0
1,-,-,ENGLAND,Diagnostic Ultrasonography,11439175.0,15.0,0.0,15.0
2,-,-,ENGLAND,Fluoroscopy,959480.0,0.0,0.0,0.0
3,-,-,ENGLAND,Magnetic Resonance Imaging,4837820.0,21.0,4.0,25.0
4,-,-,ENGLAND,Medical Photography,67400.0,0.0,0.0,0.0
5,-,-,ENGLAND,Nuclear Medicine Procedure,321835.0,22.0,1.0,23.0
6,-,-,ENGLAND,Plain Radiography,23346900.0,0.0,1.0,1.0
7,-,-,ENGLAND,Positron Emission Tomography,299550.0,9.0,2.0,11.0
8,-,-,ENGLAND,Single Photon Emission Computerized Tomography,58620.0,19.0,2.0,21.0
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
